In [1]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.6.0


In [2]:
import pandas as pd
#subscription_id = '6a0ec27b-1203-46fe-ade0-8c3003d5e4bc'
#resource_group  = 'MLServiceWrokspace'
#workspace_name  = 'MLServiceWrokspace'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Library configuration succeeded')
#except:
#    print('Workspace not found')


ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-classification-ccard-local'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,6a0ec27b-1203-46fe-ade0-8c3003d5e4bc
Workspace,MLServiceWrokspace
Resource Group,MLServiceWrokspace
Location,northeurope
Experiment Name,automl-classification-ccard-local


In [3]:
experiment_name = 'autoMLdemo1'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.externals import joblib
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import logging
from matplotlib import pyplot as plt
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
#from azureml.explain.model._internal.explanation_client import ExplanationClient

irisds = load_iris()

iris = sns.load_dataset("iris")
type(iris)

pandas.core.frame.DataFrame

In [7]:
#y = iris.species
#X = iris.drop('species',axis=1)
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                    test_size=0.3, 
#                                                    random_state=100, 
#                                                    stratify=y)


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(iris, test_size=0.2, random_state=42, shuffle=True, stratify=iris["species"])

In [12]:
##If we were not using 'preprocess = True' then we would have to convert to a numeric 

#Create a function to convert species to numeric
#def func(x):
#     if x == 'setosa':
#          return 1
#     elif x == 'versicolor':
#         return 2
#    elif x == 'virginica':
#          return 3

#from numpy import vectorize
#vfunc = vectorize(func) #Define a vectorized function which takes a nested sequence of objects or numpy arrays as inputs

#yfactTrain = vfunc(y_train)
#yfactTest = vfunc(y_test)

#yfactTrain = yfactTrain.reshape(len(yfactTrain))
#yfactTest = yfactTest.reshape(len(yfactTest))


In [9]:
#Make sure all the features are floats
#X_train = X_train.astype(np.float)
#X_test = X_test.astype(np.float)

In [9]:
print(train.head())
print(train.count())

     sepal_length  sepal_width  petal_length  petal_width     species
8    4.4           2.9          1.4           0.2          setosa    
106  4.9           2.5          4.5           1.7          virginica 
76   6.8           2.8          4.8           1.4          versicolor
9    4.9           3.1          1.5           0.1          setosa    
89   5.5           2.5          4.0           1.3          versicolor
sepal_length    120
sepal_width     120
petal_length    120
petal_width     120
species         120
dtype: int64


In [14]:
test.head()

,sepal_length,sepal_width,petal_length,petal_width,species
38,4.4,3.0,1.3,0.2,setosa
127,6.1,3.0,4.9,1.8,virginica
57,4.9,2.4,3.3,1.0,versicolor
93,5.0,2.3,3.3,1.0,versicolor
42,4.4,3.2,1.3,0.2,setosa


In [ ]:
#! pip install azureml-train-automl-runtime

In [10]:
automl_settings = {
    "n_cross_validations": 3,
    #"experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'accuracy',
                             model_explainability=True,
                             iterations = 4,
                             debug_log = 'automl_errors.log',
                             training_data = train,
                             label_column_name = 'species',
                             **automl_settings
                            )

In [11]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_68d4a055-61f1-497e-97c0-0055378cf64d

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in

In [42]:
#from azureml.train.automl import AutoMLConfig
#from azureml.train.automl.constants import Metric

#automl_settings = {
#    #"n_cross_validations": 3,
#    #"primary_metric": 'average_precision_score_weighted',
#    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
#    "verbosity": logging.INFO,
#    "enable_stack_ensemble": False
#}

#Automl_config = AutoMLConfig(task = 'classification',
#                             debug_log = 'automl_errors.log',
#                             primary_metric = Metric.Accuracy,
#                             model_explainability=True, #make the models explainable
#                             iteration_timeout_minutes = 12000,
#                             iterations = 6,
#                             featurization = "auto",
#                             training_data = train,
#                             label_column_name = 'species',
#                             validation_data = test,
#                             **automl_settings)

##Local compute 
#Automl_config = AutoMLConfig(task = 'classification',
#                             primary_metric = Metric.Accuracy,
#                             #primary_metric = Metric.AUCWeighted,
#                             model_explainability=True, #make the models explainable
#                             iteration_timeout_minutes = 12000,
#                             iterations = 6,
#                             #n_cross_validations = 2,
#                             featurization = "auto",
#                             #exit_score = 0.995,
#                             #blacklist_models = ['kNN','LinearSVM'],
#                             X = X_train,
#                             y = yfactTrain,
#                             X_valid = X_test,
#                             y_valid = yfactTest,
#                             path='./sample_projects/automl-local-classification')

In [12]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
#Breakdown of the results
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

import pandas as pd
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1
AUC_macro,0.98,0.99
AUC_micro,0.99,1.00
AUC_weighted,0.98,0.99
accuracy,0.95,0.96
average_precision_score_macro,0.98,0.99
average_precision_score_micro,0.98,0.99
average_precision_score_weighted,0.98,0.99
balanced_accuracy,0.95,0.96
f1_score_macro,0.95,0.96
f1_score_micro,0.95,0.96


In [14]:
#Get the best model
best_run, fitted_model = local_run.get_output()


In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
autoMLdemo1,AutoML_68d4a055-61f1-497e-97c0-0055378cf64d_1,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [66]:
#Test with the testing data

from sklearn.metrics import confusion_matrix, roc_curve, auc

y_pred = (fitted_model.predict(test.iloc[:,0:4]))

accuracy_score(test.iloc[:,4], y_pred)

cm = confusion_matrix(test.iloc[:,4], y_pred)
print(cm)


[[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]
